In [1]:
! pip install numpy
! pip install matplotlib
! pip install scikit-learn
! pip install pandas
! pip install tensorflow


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
from PIL import Image

In [3]:
import os

In [4]:
from tensorflow import keras

In [5]:
train = keras.utils.image_dataset_from_directory(
    directory='data', 
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256),
    validation_split = 0.2,
    subset = 'training',
    seed = 123
)

validate = keras.utils.image_dataset_from_directory(
    directory='data', 
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256),
    validation_split = 0.2, 
    subset = 'validation',
    seed = 123
)

Found 24959 files belonging to 2 classes.
Using 19968 files for training.
Found 24959 files belonging to 2 classes.
Using 4991 files for validation.


In [6]:
def img_processing(image, label):
    image = tf.cast(image/255., tf.float64)
    return image, label

In [7]:
train = train.map(img_processing)
validate = validate.map(img_processing)

In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

In [11]:
model = Sequential()

model.add(Conv2D(
    filters = 32,
    kernel_size = (3,3),
    activation=tf.keras.activations.relu,
    input_shape=(256,256,3)
))
model.add(MaxPooling2D(
    pool_size=(2, 2),
    strides = 2,
))
model.add(Conv2D(
    filters = 64,
    kernel_size = (3,3),
    activation=tf.keras.activations.relu,
))
model.add(MaxPooling2D(
    pool_size=(2, 2),
    strides = 2,
))
model.add(Conv2D(
    filters = 128,
    kernel_size = (3,3),
    activation=tf.keras.activations.relu,
))
model.add(MaxPooling2D(
    pool_size=(2, 2),
    strides = 2,
))
model.add(Flatten())
model.add(Dense(
    units = 128,
    activation = tf.keras.activations.relu,
))
model.add(Dropout(0.2))
model.add(Dense(
    units = 64,
    activation = tf.keras.activations.relu,
))
model.add(Dropout(0.2))
model.add(Dense(
    units = 1,
    activation = tf.keras.activations.sigmoid
))

In [12]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,847,297 (56.64 MB)

 Trainable params: 14,847,297 (56.64 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.2), loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(train, epochs=10, validation_data = validate)

Epoch 1/10
101/624 ━━━━━━━━━━━━━━━━━━━━ 4:36 528ms/step - accuracy: 0.4963 - loss: 0.7119

InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Input size should match (header_size + row_size * abs_height) but they differ by 2
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_multi_step_on_iterator_2875]